In [0]:
# Load it into a DataFrame
df = spark.read.csv("/databricks-datasets/samples/population-vs-price/data_geo.csv", header=True, inferSchema=True)
df.show(5)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+---------+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
df.shape()


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-927143160712520>:1
----> 1 df.shape()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:2964, in DataFrame.__getattr__(self, name)
   2934 """Returns the :class:`Column` denoted by ``name``.
   2935 
   2936 .. versionadded:: 1.3.0
   (...)
   2961 +---+
   2962 """
   2963 if name not in self.columns:
-> 2964     raise AttributeError(
   2965         "'%s' object has no attribute '%s'" % (self.__class__.__name__, name)
   2966     )
   2967 jc = self

In [0]:
file_path = "/databricks-datasets/samples/population-vs-price/data_geo.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  30101

In [0]:
# Select relevant columns for analysis
df_selected = df.select(
    "City", "State", "State Code", 
    "2014 Population estimate", "2015 median sales price"
)
df_selected.show(5)

+-------------+-------+----------+------------------------+-----------------------+
|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+-------------+-------+----------+------------------------+-----------------------+
|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|       Mobile|Alabama|        AL|                  194675|                  122.5|
|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
df_selected.orderBy("2014 Population estimate", ascending=False).show(10)


+---------------+------------+----------+------------------------+-----------------------+
|           City|       State|State Code|2014 Population estimate|2015 median sales price|
+---------------+------------+----------+------------------------+-----------------------+
|    New York[6]|    New York|        NY|                 8491079|                  388.6|
|    Los Angeles|  California|        CA|                 3928864|                  434.7|
|        Chicago|    Illinois|        IL|                 2722389|                  192.5|
|     Houston[7]|       Texas|        TX|                 2239558|                  200.3|
|Philadelphia[8]|Pennsylvania|        PA|                 1560297|                  204.9|
|        Phoenix|     Arizona|        AZ|                 1537058|                  206.1|
|    San Antonio|       Texas|        TX|                 1436697|                  184.7|
|      San Diego|  California|        CA|                 1381069|                  510.3|

In [0]:
# Step 1: Get top 10 most populous cities
top10_populous = df_selected.orderBy("2014 Population estimate", ascending=False).limit(10)

# Step 2: Display the result (Databricks will allow you to switch to a chart view)
display(top10_populous)


City,State,State Code,2014 Population estimate,2015 median sales price
New York[6],New York,NY,8491079,388.6
Los Angeles,California,CA,3928864,434.7
Chicago,Illinois,IL,2722389,192.5
Houston[7],Texas,TX,2239558,200.3
Philadelphia[8],Pennsylvania,PA,1560297,204.9
Phoenix,Arizona,AZ,1537058,206.1
San Antonio,Texas,TX,1436697,184.7
San Diego,California,CA,1381069,510.3
Dallas,Texas,TX,1281047,192.5
San Jose,California,CA,1015785,900.0


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import avg
df_average=df_selected.groupBy("State").agg(avg("2015 median sales price")).show()
display(df_average)

+--------------------+----------------------------+
|               State|avg(2015 median sales price)|
+--------------------+----------------------------+
|                Utah|                       243.3|
|             Hawai'i|                       699.3|
|           Minnesota|                       209.4|
|                Ohio|          112.86666666666667|
|            Arkansas|                       131.8|
|              Oregon|          231.53333333333333|
|               Texas|           173.4777777777778|
|        North Dakota|                       180.0|
|        Pennsylvania|                       183.8|
|         Connecticut|          250.13333333333333|
|            Nebraska|          149.14999999999998|
|              Nevada|                       237.3|
|          Washington|          266.29999999999995|
|            Illinois|          122.87499999999999|
|            Oklahoma|                       145.1|
|District of Columbia|                       367.8|
|           

In [0]:
from pyspark.sql.functions import avg
df_average=df_selected.groupBy("State").agg(avg("2015 median sales price"))
display(df_average)

State,avg(2015 median sales price)
Utah,243.3
Hawai'i,699.3
Minnesota,209.4
Ohio,112.86666666666667
Arkansas,131.8
Oregon,231.53333333333333
Texas,173.4777777777778
North Dakota,180.0
Pennsylvania,183.8
Connecticut,250.13333333333333


Databricks visualization. Run in Databricks to view.

In [0]:
filtered_df = df_selected.filter(df_selected["2014 Population estimate"] > 1000000)

# Get top 10 based on population
top10_million_plus = filtered_df.orderBy("2014 Population estimate", ascending=False).limit(10)

# Display the result in Databricks
display(top10_million_plus)

City,State,State Code,2014 Population estimate,2015 median sales price
New York[6],New York,NY,8491079,388.6
Los Angeles,California,CA,3928864,434.7
Chicago,Illinois,IL,2722389,192.5
Houston[7],Texas,TX,2239558,200.3
Philadelphia[8],Pennsylvania,PA,1560297,204.9
Phoenix,Arizona,AZ,1537058,206.1
San Antonio,Texas,TX,1436697,184.7
San Diego,California,CA,1381069,510.3
Dallas,Texas,TX,1281047,192.5
San Jose,California,CA,1015785,900.0


Databricks visualization. Run in Databricks to view.

In [0]:
df_new = df_selected.dropna()
display(df_new)

City,State,State Code,2014 Population estimate,2015 median sales price
Birmingham,Alabama,AL,212247,162.9
Huntsville,Alabama,AL,188226,157.7
Mobile,Alabama,AL,194675,122.5
Montgomery,Alabama,AL,200481,129.0
Phoenix,Arizona,AZ,1537058,206.1
Tucson,Arizona,AZ,527972,178.1
Little Rock,Arkansas,AR,197706,131.8
Anaheim,California,CA,346997,685.7
Los Angeles,California,CA,3928864,434.7
Riverside,California,CA,319504,281.0
